In [33]:
import sqlite3
import pandas as pd

In [40]:
def get_navbar_information():
    # Mở kết nối đến cơ sở dữ liệu SQLite
    conn = sqlite3.connect('Database/database.db')
    conn.row_factory = sqlite3.Row  # Thiết lập row_factory để trả về từ điển thay vì tuple
    cursor = conn.cursor()
    
    # Lấy thông tin user_id từ bảng users (đây là một ví dụ, bạn cần điều chỉnh cho phù hợp với cấu trúc của cơ sở dữ liệu của bạn)
    # cursor.execute("SELECT user_id FROM users WHERE ...")
    # user_id = cursor.fetchone()[0]  # Giả sử user_id là cột đầu tiên trong kết quả

    if False: ## Chưa đăng nhập
        user_id = 0
    else:
        user_id = 0

    # Truy vấn để lấy thông tin giỏ hàng dựa trên user_id
    cursor.execute("SELECT * FROM Cart WHERE user_id IS ? OR user_id = ?", (user_id, user_id))
    cart = cursor.fetchone()

    # Chuyển đổi sqlite3.Row thành dictionary
    cart = dict(cart) if cart else None

    total_quantity = 0
    total_amount = 0

    # Lấy thông tin về các mặt hàng trong giỏ hàng dựa vào cart['id']
    if cart:
        # Nếu có giỏ hàng tương ứng, truy vấn để lấy thông tin chi tiết của các mặt hàng trong giỏ hàng dựa trên cart_id và product_id
        cursor.execute("""
            SELECT products.*, CartItems.quantity
            FROM CartItems
            INNER JOIN Products ON CartItems.product_id = products.id
            WHERE CartItems.cart_id = ?
        """, (cart['id'],))
        cart_items = [dict(row) for row in cursor.fetchall()]

        # Tính tổng số lượng hàng và tổng số tiền
        for item in cart_items:
            total_quantity += item['quantity']
            total_amount += item['quantity'] * item['price']

        # Cập nhật giá trị total_quantity và total_amount vào bảng Cart
        cursor.execute("""
            UPDATE cart
            SET total_quantity = ?,
                total_amount = ?
            WHERE id = ?
        """, (total_quantity, total_amount, cart['id']))
        conn.commit()

    # Đóng kết nối đến cơ sở dữ liệu
    conn.close()

    # Trả về thông tin đã thu thập được
    return user_id, cart, cart_items
    

In [42]:
user_id, cart, cart_items = get_navbar_information()

for item in cart_items:
    print(item)

{'id': 1, 'name': 'Gift Card', 'url': 'gift-card', 'category': 'Gift Cards', 'description': 'Description of Product 1', 'price': 25.99, 'discount': 0.0, 'bought': 10.0, 'inventory': 100.0, 'image_url': 'https://assets-global.website-files.com/6644881a99e1e50c056f0640/6644881a99e1e50c056f06b4_acme-gift-card.jpg', 'quantity': 2}
{'id': 3, 'name': 'Blue Canvas Pack', 'url': 'blue-canvas-pack', 'category': 'Packs', 'description': 'Description of Product 3', 'price': 95.0, 'discount': 50.0, 'bought': 30.0, 'inventory': 300.0, 'image_url': 'https://assets-global.website-files.com/6644881a99e1e50c056f0640/6644881a99e1e50c056f0669_denisse-leon-J7CjWufjmg4-unsplash.jpg', 'quantity': 1}


In [36]:
# Kết nối đến database
conn = sqlite3.connect('database/database.db')

# Đọc dữ liệu từ bảng Products vào DataFrame
df = pd.read_sql_query("SELECT * FROM Products", conn)

# Đóng kết nối
conn.close()

df

,id,name,url,category,description,price,discount,bought,inventory,image_url
0,1,Gift Card,gift-card,Gift Cards,Description of Product 1,25.99,0.0,10.0,100.0,https://assets-global.website-files.com/664488...
1,2,Tin Coffee Tumbler,tin-coffee-tumbler,Accessories,Description of Product 2,35.99,0.0,20.0,200.0,https://assets-global.website-files.com/664488...
2,3,Blue Canvas Pack,blue-canvas-pack,Packs,Description of Product 3,95.00,50.0,30.0,300.0,https://assets-global.website-files.com/664488...
3,4,Green Canvas Pack,green-canvas-pack,Packs,Description of Product 4,39.99,0.0,30.0,300.0,https://assets-global.website-files.com/664488...
4,5,Product 5,product-5,Tents,Description of Product 5,200.99,0.0,100.0,1000.0,https://assets-global.website-files.com/664488...


In [37]:
# Kết nối đến database
conn = sqlite3.connect('database/database.db')

# Đọc dữ liệu từ bảng Products vào DataFrame
df = pd.read_sql_query("SELECT * FROM Cart", conn)

# Đóng kết nối
conn.close()

df

,id,user_id,total_quantity,total_amount,created_at,updated_at
0,1,0,3,146.98,2024-05-21 06:57:37,2024-05-21 06:57:37


In [38]:
# Kết nối đến database
conn = sqlite3.connect('database/database.db')

# Đọc dữ liệu từ bảng Products vào DataFrame
df = pd.read_sql_query("SELECT * FROM CartItems", conn)

# Đóng kết nối
conn.close()

df

,id,cart_id,product_id,quantity,price
0,1,1,1,2,51.98
1,2,1,3,1,95.00
